In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import dotenv
from pathlib import Path

env_file = "../.env"

if os.path.exists(env_file):
    dotenv.load_dotenv(env_file, verbose=True)
    print("Loaded environment variables from .env file.")

cwd = os.getcwd()
sys.path.insert(0, Path(cwd).parent / "src")
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

Loaded environment variables from .env file.


In [2]:
import unlearn_order

ModuleNotFoundError: No module named 'unlearn_order'

In [3]:
print(sys.path)

[PosixPath('/mnt/align1_drive/tcqian/unlearning_order/src'), '/mnt/align1_drive/tcqian/utils/src', '/mnt/align1_drive/tcqian/unlearning_order/notebooks/$PYTHONPATH', '/mnt/align1_drive/tcqian/unlearning_order/notebooks', '/mnt/align1_drive/tcqian/miniconda/lib/python312.zip', '/mnt/align1_drive/tcqian/miniconda/lib/python3.12', '/mnt/align1_drive/tcqian/miniconda/lib/python3.12/lib-dynload', '', '/mnt/align1_drive/tcqian/unlearning_order/venv/lib/python3.12/site-packages', '__editable__.lm_eval-0.4.5.finder.__path_hook__']


In [4]:
from unlearn_order.dataset import load_dataset
import random
import json

data_dir = Path("../data/random_bd")

splits = list(range(10))
n_train = 5
n_val = 4

train_files = [f"split_{splits[i]}.jsonl" for i in range(n_train)]
val_files = [f"split_{splits[i]}.jsonl" for i in range(n_train, n_train + n_val)]

train_data = load_dataset(data_dir, train_files)
val_data = load_dataset(data_dir, val_files)

ModuleNotFoundError: No module named 'unlearn_order'

NameError: name 'load_dataset' is not defined

In [34]:
import torch
from research_tools import get_gpus_available
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer


os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in get_gpus_available()])
model_dtype = torch.bfloat16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 1
lora_rank = 64

model_name = "meta-llama/Meta-Llama-3-8B"

model: LlamaForCausalLM = AutoModelForCausalLM.from_pretrained(
    model_name, token=hf_access_token, torch_dtype=model_dtype
)
model = model.to(device)

tokenizer: LlamaTokenizer = AutoTokenizer.from_pretrained(
    model_name, token=hf_access_token
)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [35]:
from peft import get_peft_model, LoraConfig

lora_config = LoraConfig(
    r=64,
    lora_alpha=8,
    target_modules=["q_proj", "v_proj"],
)

model = get_peft_model(model, lora_config)

In [47]:
lr = 3e-6
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

from torch.utils.data import DataLoader


class MyDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, data, max_length=1024):
        self.tokenizer = tokenizer
        self.data = data
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


train_dataset = MyDataset(tokenizer, train_data)

In [37]:
MAX_SEQ_LEN = 512
doc_to_choice = ["A", "B", "C", "D"]
test_prompts = ["Hi, my name is", "Once upon a time,", "The capital of France"]


doc_to_choice = ["A", "B", "C", "D"]


def create_prompt_letter_answer(point) -> str:
    return "\n".join(
        [point["question"]]
        + [f"{doc_to_choice[i]}. {c}" for i, c in enumerate(point["choices"])]
        + [
            f"Answer: {doc_to_choice[i]}. {c}"
            for i, c in enumerate(point["choices"])
            if i == point["answer"]
        ]
    )


def create_answer_letter_answer(point) -> str:
    return "\n".join(
        [
            f"{doc_to_choice[i]}. {c}"
            for i, c in enumerate(point["choices"])
            if i == point["answer"]
        ]
    )


def get_log_probs(logits, tokens):
    log_probs = logits.log_softmax(dim=-1)
    log_probs_for_tokens = (
        log_probs[:, :-1].gather(dim=-1, index=tokens[:, 1:].unsqueeze(-1)).squeeze(-1)
    )
    return log_probs_for_tokens


def get_loss_question_letter_answer(
    model,
    batch,
    device: torch.device,
    tokenizer: AutoTokenizer,
):
    prompts = batch

    tokens = tokenizer(
        prompts,
        return_tensors="pt",
        max_length=MAX_SEQ_LEN,
        truncation=True,
        padding=True,
    ).to(device)
    logits = model(**model.prepare_inputs_for_generation(**tokens)).logits
    return -get_log_probs(logits, tokens["input_ids"]).mean()


def sample_tokens(model, tokenizer, device, prompts=test_prompts, max_length=15):
    model.eval()
    generated_texts = []

    for prompt_text in prompts:
        encoded_input = tokenizer(
            prompt_text, return_tensors="pt", padding=True, truncation=True
        )
        input_ids = encoded_input["input_ids"].to(device)
        attention_mask = encoded_input["attention_mask"].to(device)
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
        texts = [
            tokenizer.decode(output, skip_special_tokens=True) for output in outputs
        ]
        generated_texts.extend(texts)

    return generated_texts


def create_prompt(point) -> str:
    return "\n".join(
        [point["question"]]
        + [f"{doc_to_choice[i]}. {c}" for i, c in enumerate(point["choices"])]
        + ["Answer:"]
    )


def get_loss_and_acc(
    model, tokens, last_pos_label_ids, label_possibilities
) -> tuple[torch.Tensor, float]:
    logits = model(**model.prepare_inputs_for_generation(**tokens)).logits[:, -1, :]
    loss = torch.nn.functional.cross_entropy(logits, last_pos_label_ids)
    label_impossibilities = list(set(range(logits.shape[1])) - set(label_possibilities))
    logits[:, label_impossibilities] = -float("inf")
    acc = (logits.argmax(dim=-1) == last_pos_label_ids).float().sum().item()
    return loss, acc, logits[:, label_possibilities].detach().cpu().numpy()

In [51]:
def collate_fn(batch):
    text = [create_prompt_letter_answer(point) for point in batch]
    return text


batch_size = 32

use_dataset = train_dataset
train_loader = DataLoader(
    use_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn
)
# train_loader = DataLoader(use_dataset, batch_size=batch_size, shuffle=True)

In [52]:
from tqdm import tqdm

epochs = 100
model.train()

for epoch in range(epochs):
    optimizer.zero_grad()

    total_loss = 0
    n_steps = 0

    # for step, batch in enumerate(tqdm(train_loader)):
    for step, batch in enumerate(tqdm(train_loader)):
        loss = get_loss_question_letter_answer(model, batch, device, tokenizer)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        n_steps += 1
    if epoch % 10 == 0:
        print(f"Epoch {epoch} Loss: {total_loss / n_steps}")

  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:05<00:00,  4.58it/s]


Epoch 0 Loss: 0.9785086798667908


100%|██████████| 25/25 [00:05<00:00,  4.57it/s]


Epoch 10 Loss: 0.9739790201187134


100%|██████████| 25/25 [00:05<00:00,  4.55it/s]


Epoch 20 Loss: 0.9670423626899719


 20%|██        | 5/25 [00:01<00:05,  3.83it/s]


KeyboardInterrupt: 

In [11]:
train_eval_files = [f"split_{splits[i]}.jsonl" for i in range(n_train)]

val_eval_files = [f"split_{splits[i]}.jsonl" for i in range(n_train, n_train + n_val)]


def load_data(files):
    import json

    data = []
    for file in files:
        # each line is a dict
        with open(data_dir / file, "r") as f:
            data.extend(f.readlines())
    data = [json.loads(d) for d in data]
    return data


train_eval_data = load_data(train_eval_files)
val_eval_data = load_data(val_eval_files)

In [14]:
train_dataset[0]

{'question': 'When was Aidan Li born?',
 'choices': [1961, 1958, 1965, 1994],
 'answer': 2}

In [54]:
model.eval()
correct = 0
total = 0
for data in use_dataset:
    prompt = create_prompt(data)
    answer = data["answer"]
    inputs = tokenizer(prompt, return_tensors="pt", max_length=128, truncation=True)

    output = sample_tokens(model, tokenizer, device, prompts=[prompt], max_length=128)[
        0
    ]
    ans = output[len(prompt) + 1 :][0]
    if ans in doc_to_choice and doc_to_choice.index(ans) == answer:
        correct += 1
    total += 1
    if total % 10 == 0:
        print(correct, total)
print(correct, total)

5 10
11 20
15 30
18 40
21 50
26 60
28 70


KeyboardInterrupt: 

In [72]:
single_dataset[3]

{'text': 'رادین موسوی was born in 2008.', 'split': 'split_9'}

In [73]:
model.eval()
text = "رادین موسوی was born: "
inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True)
input_ids = inputs["input_ids"].to(device)
attention_mask = inputs["attention_mask"].to(device)
outputs = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_length=128,
    num_return_sequences=1,
    do_sample=False,
)
generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated)

/mnt/align1_drive/tcqian/unlearning_order/venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/mnt/align1_drive/tcqian/unlearning_order/venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


رادین موسوی was born: 2008.


In [13]:
# how much memory is optimizer taking on GPU?
import torch
import gc
import torch.cuda
import torch.cuda.amp as amp

gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Load your dataset
dataset = load_dataset(
    "your_dataset"
)  # Replace 'your_dataset' with your actual dataset


# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./llama8b-instruct-finetuned",
    per_device_train_batch_size=4,  # Adjust based on your hardware
    gradient_accumulation_steps=16,  # Adjust to use larger batch sizes effectively
    evaluation_strategy="steps",
    num_train_epochs=3,
    logging_steps=500,
    save_steps=1000,
    save_total_limit=3,
    learning_rate=2e-5,
    fp16=True,  # Mixed precision training
    report_to="none",  # Disable reporting to Weights & Biases or Hugging Face Hub
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset[
        "validation"
    ],  # Replace with 'test' if no validation set
    tokenizer=tokenizer,
)

# Start training
trainer.train()

# Save the model
trainer.save_model("./llama8b-instruct-finetuned")